In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# from main import get_normalized_probability, cal_most_prob_scene, reverse_relation
import json
import pulp
import numpy as np
from LPConstraints import get_clevr_block_world_constraint
from utils import process_coord_scene

In [143]:
with open('../results/clevr_block_val/constraint_1234/rel_scenes.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('../data/clevr_block/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [110]:
scenes = [process_coord_scene(scene, 'blockworld') for scene in scenes]

In [19]:
def get_attribute_variables(objects, attributes_map):
    variables = []
    probabilities = {}
    num_objects = len(objects)

    for name, values in attributes_map.items():
        for v, _ in enumerate(values):
            for i in range(num_objects):
                variable_name = f'{name}_{i}_{v}'
                variables.append(variable_name)
                probabilities[variable_name] = objects[i][name][v]
    return variables, probabilities

def get_relationship_variables(scene, relationships):
    variables = []
    probabilities = {}

    for rel in relationships:
        for target, sources in enumerate(scene['relationships'][rel]):
            for source, probability in sources:
                # TODO: change the prediction for below and above (currently flipped)
                variable_name = f'{rel}_{source}_{target}'
                variables.append(variable_name)
                probabilities[variable_name] = probability
    return variables, probabilities

def get_LP_problem(scene, attr_map, eps=1e-50):
    attr_variables, attr_probabilities = get_attribute_variables(scene['objects'], attr_map['attributes'])
    rel_variables, rel_probabilities = get_relationship_variables(scene, attr_map['relations'])


    attr_variables = pulp.LpVariable.dict('attr', attr_variables, 0, 1, pulp.LpBinary)
    rel_variables = pulp.LpVariable.dict('rel', rel_variables, 0, 1, pulp.LpBinary)

    attr_objective = [attr_variables[i] * np.log(max(attr_probabilities[i], eps)) for i in attr_variables.keys()]
    rel_objective = [rel_variables[i] * np.log(max(rel_probabilities[i], eps)) + (1 - rel_variables[i]) * np.log((max(1 - rel_probabilities[i], eps))) for i in rel_variables.keys()]

    prob = pulp.LpProblem("sceneGraphProblem", pulp.LpMaximize)
    prob += pulp.lpSum(attr_objective + rel_objective)
    return prob, attr_variables, rel_variables

def get_predicted_scene_for_block_world(variables, scene):
    predicted_scene = {
        'objects': [{'mask': o['mask']} for o in scene['objects']],
        'relationships': { rel: [[] for _ in range(len(scene['objects']))] for rel in attr_map['relations']}
    }

    for v in variables:
        if v.varValue:
            tokens = v.name.split('_')
            if tokens[0] == 'attr':
                predicted_scene['objects'][int(tokens[2])][tokens[1]] = attr_map['attributes'][tokens[1]][int(tokens[3])]
            elif tokens[0] == 'rel':
                predicted_scene['relationships'][tokens[1]][int(tokens[3])].append(int(tokens[2]))
    return predicted_scene

In [113]:
from tqdm import tqdm
M = 100
predicted_scenes = []
solver = pulp.GUROBI_CMD(path = 'C:/gurobi951/win64/bin/gurobi_cl.exe')

for scene in tqdm(scenes):
    prob, attr_variables, rel_variables = get_LP_problem(scene, attr_map)
    get_clevr_block_world_constraint(prob, scene, rel_variables, attr_variables, attr_map, include_small_obj=True, include_large_cube=True, include_bottom_stack=True, include_yellow=True)
    prob.solve(solver)
    predicted_scenes.append(get_predicted_scene_for_block_world(prob.variables(), scene))

100%|██████████| 6000/6000 [10:03<00:00,  9.94it/s]


In [114]:
with open('../results/clevr_block_val/constraint_1234/scene_prob_fix.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)



Experiment with Gurobi Constraint formulations

In [3]:
from LPProblem import BlocksworldLPProblem

In [4]:
with open('../results/clevr_block_val/constraint_1234/scenes_coord_prob.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('../data/clevr_block/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [5]:
scenes = [process_coord_scene(scene, 'blockworld') for scene in scenes]

In [6]:
problem_solver = BlocksworldLPProblem(attr_map, include_small_obj=True, include_large_cube=True, include_bottom_stack=True, include_yellow=True)

In [11]:
predicted_scenes = []
for scene in tqdm(scenes):
    predicted_scenes.append(problem_solver.solve_for_scene(scene))

  0%|          | 0/6000 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-27


 12%|█▏        | 746/6000 [00:25<03:02, 28.83it/s]


KeyboardInterrupt: 

In [169]:
with open('../results/clevr_block_val/constraint_1234/rel_scenes_fix.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)

Experiment with Clevr

In [70]:
from LPProblem import ClevrLPProblem
from tqdm import tqdm
import json
from utils import process_coord_scene_blockworld_clevr

with open('../results/clevr_val/c3/rel_scenes.json', 'r') as f:
    scenes = json.load(f)['scenes']

with open('G:/My Drive/Research/data/Clevr_mini/clevr_attr_map.json', 'r') as f:
    attr_map = json.load(f)

In [71]:
scenes = [process_coord_scene_blockworld_clevr(scene) for scene in scenes]


KeyError: 'x'

In [72]:
problem_solver = ClevrLPProblem(attr_map,color_between=True, material_order=True, large_cube=True, object_behind=True)

In [73]:
predicted_scenes = []
for scene in tqdm(scenes):
    predicted_scenes.append(problem_solver.solve_for_scene(scene))

100%|██████████| 1000/1000 [00:49<00:00, 20.25it/s]


In [74]:
with open('../results/clevr_val/c3/rel_scenes_fix_c1234.json', 'w') as f:
    json.dump({'scenes': predicted_scenes}, f)

In [60]:
for scene in tqdm(scenes[21:22]):
    scene = problem_solver.solve_for_scene(scene)

100%|██████████| 1/1 [00:00<00:00, 62.67it/s]


In [61]:
scene

{'objects': [{'mask': {'size': [320, 480],
    'counts': 'f^W2h0W92O0000000O2O00001N10001N10000O100001O00001O00001N10000O2O00000O10[mS2'},
   'shape': 'cube',
   'size': 'small',
   'material': 'metal',
   'color': 'cyan'},
  {'mask': {'size': [320, 480],
    'counts': '[\\i1=`94M3N1O1O2N1O1O100O2O1O1N3N2N1O000000000000000001O000O2O0O3M1O2N2N3M2M4LiQa2'},
   'shape': 'sphere',
   'size': 'large',
   'material': 'metal',
   'color': 'red'},
  {'mask': {'size': [320, 480],
    'counts': 'o\\d1?_9<E3N1N101O000O10000000000000000000000001O00000O100O2OTkh2'},
   'shape': 'cylinder',
   'size': 'small',
   'material': 'metal',
   'color': 'cyan'}],
 'relationships': {'left': [[1, 2], [2], []],
  'right': [[], [0], [0, 1]],
  'front': [[2], [0, 2], []],
  'behind': [[1], [], [0, 1]]}}

In [231]:
model.write("file.lp")